#Import pandas packages & read data

In [5446]:
import pandas as pd
import numpy as np
df = pd.read_excel("C:/Users/prana/Desktop/DBMS/Project/Amtrak-Original File.xlsx",sheet_name=None)

#State Table

In [5448]:
State = pd.DataFrame( df['Station'])  #main station tab

State['stateCode'] = State['City'].astype(str).str.split(',').str[1].str[:3]  #fetch state code from city after splitting
State = State.drop_duplicates(['State','stateCode']).loc[:,['State','stateCode']]  #drop state name duplicates
State = State[State['State'].isna() ==False]  #filter rows with no state values

State['State'] = State['State'].str.rstrip().str.lstrip().drop_duplicates()  #remove white spaces
State['stateCode'] = State['stateCode'].str.lstrip().str.rstrip()  #remove white spaces
State = State[['stateCode','State']]  #re-arrange columns
State.columns = ['stateCode','stateName']  #rename columns

State.head()

,stateCode,stateName
0,AL,Alabama
3,AZ,Arizona
10,AR,Arkansas
16,BC,British Columbia (Canada)
17,CA,California


#Changes to original dataframes

In [5450]:
df['Station']['State'] = df['Station']['State'].fillna(method='ffill')  #fillNA values
df['Station']['State'] = df['Station']['State'].str.lstrip().str.rstrip()  #remove white spaces
df['Station']['City'] = df['Station']['City'].str.lstrip().str.rstrip()
df['Station'].head()

df['Budget (in$K)']['Station Name'] = df['Budget (in$K)']['Station Name'].str.lstrip().str.rstrip()  #remove white spaces
df['Budget (in$K)']['stationStateCode'] = df['Budget (in$K)']['Station Name'].str[-2:] #fetch state code from station name
df['Budget (in$K)']['Plan Year'] = df['Budget (in$K)']['Plan Year'].str.replace("FY","20").str.replace('PP','2024') #replace with year numbers
df['Budget (in$K)']['Budget'] = df['Budget (in$K)']['Budget'].astype(int)*1000  #multiply with 1000 to get actual dollar amount
df['Budget (in$K)'].head()

df['Ridership']['State'] = df['Ridership']['State'].fillna(method='ffill').str.lstrip().str.rstrip()  #fillna state values
df['Ridership'] = df['Ridership'] [ df['Ridership']['City (Code) '].isna() == False ]  #remove N/As since we only want to fetch station level values
df['Ridership'] = df['Ridership'][ df['Ridership']['City (Code) '].astype(str).str.contains('Total') == False ]  #drop data with total in the string
df['Ridership']['City (Code) '] = df['Ridership'] ['City (Code) '].str.lstrip().str.rstrip()  #remove white spaces
df['Ridership']['Code'] = df['Ridership']['City (Code) '].astype(str).str.extract(r'\((\w{3})\)')  #extract 3 letter code
df['Ridership']['City (Code) '] = df['Ridership']['City (Code) '].str[:-6]  #remove city code
df['Ridership'] = pd.merge(df['Ridership'],State,left_on = 'State',right_on = 'stateName',how = 'left')  #fetch state key
df['Ridership'] ['City (Code) '] = df['Ridership'] ['City (Code) '] + ', ' + df['Ridership'] ['stateCode']  #concat state key to station (LAK)
df['Ridership'] = df['Ridership'][['State','stateCode','City (Code) ','Code','FY21','FY22','FY23']] #rearrange columns
df['Ridership'].head()

df['Procurement'] = df['Procurement'] [ df['Procurement']['State'].isna() == False ]  #remove N/As since we only want to fetch state level procurement value
df['Procurement']['State'] = df['Procurement']['State'].str.lstrip().str.rstrip()  #remove white spaces
df['Procurement'] = df['Procurement'].loc[:,['State','FY21','FY22','FY23']]  #drop station column
df['Procurement'].head()

df['On-Time Performance (OTP)'] = df['On-Time Performance (OTP)'] [['Type','Route','Stations','FY21 OTP','FY22 OTP','FY23 OTP']]
updated_df = []
for index, row in df['On-Time Performance (OTP)'].iterrows(): #iterate through each row
      stations = row['Stations'].split('-')  #split stations column based on the hyphen
      for station in stations:  #run for loop to ensure rest of the data remains and append to updated df
            updated_df.append({  
              "Type": row['Type'],
              "Route": row['Route'],
              "Station": station.strip(), 
              "FY21 OTP": row['FY21 OTP'],
              "FY22 OTP": row['FY22 OTP'],
              "FY23 OTP": row['FY23 OTP']
             })

df['On-Time Performance (OTP)'] = pd.DataFrame(updated_df)

df['On-Time Performance (OTP)']['Station'] = df['On-Time Performance (OTP)']['Station'].str.lstrip().str.rstrip()  #remove white spaces
df['On-Time Performance (OTP)']['Route'] = df['On-Time Performance (OTP)']['Route'].str.lstrip().str.rstrip()  
df['On-Time Performance (OTP)']['Type'] = df['On-Time Performance (OTP)']['Type'].str.lstrip().str.rstrip()  
df['On-Time Performance (OTP)'] = df['On-Time Performance (OTP)'].drop_duplicates(['Type','Route','Station']) #drop duplicates because of same data for diffferent states


df['Employment'] = df['Employment'][df['Employment']['State'].isna() == False]
df['Employment']['State'] = df['Employment']['State'].str.lstrip().str.rstrip()  #remove white spaces
df['Employment'].head()

C:\Users\prana\AppData\Local\Temp\ipykernel_21856\2972754139.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Station']['State'] = df['Station']['State'].fillna(method='ffill')  #fillNA values
C:\Users\prana\AppData\Local\Temp\ipykernel_21856\2972754139.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Ridership']['State'] = df['Ridership']['State'].fillna(method='ffill').str.lstrip().str.rstrip()  #fillna state values


,State,FY21,$FY21,FY22,$FY22,FY23,$FY23
1,Alabama,13,1292695,15,1976833,27,1983661
2,Arizona,12,1889753,15,1466911,28,2460877
3,Arkansas,26,2876205,28,3183395,31,2646581
4,California,1842,161770256,1809,175959691,2056,185233855
5,Colorado,58,5733988,65,7819320,76,8674629


#Station Table

In [5452]:
Station = pd.DataFrame(df['Station'].loc[:,['City','Code','State']])  #main stations tab
Station = pd.merge(Station,State,left_on = 'State',right_on = 'stateName',how = 'left')  #fetch state key
Station = Station.loc[:,['Code','City','stateCode']]
Station.head()

budgetStation = pd.DataFrame(df['Budget (in$K)']).loc[:,['Code','Station Name','stationStateCode']]  
budgetStation.columns = ['Code','City','stateCode']
budgetStation.head()

ridershipStation = pd.DataFrame(df['Ridership']).loc[:,['Code','City (Code) ','stateCode']]
ridershipStation.columns = ['Code','City','stateCode']
ridershipStation.head()

Station = pd.concat([Station,budgetStation,ridershipStation],axis = 0).drop_duplicates()  #fetch stations from other tabs that are not there on stations
Station.head()

Station.columns = ['stationCode','stationName','stateCode']
Station = Station[Station['stationName'].isna() == False]
Station.head()

,stationCode,stationName,stateCode
0,ATN,"Anniston, AL",AL
1,BHM,"Birmingham, AL",AL
2,TCL,"Tuscaloosa, AL",AL
3,BEN,"Benson, AZ",AZ
4,FLG,"Flagstaff, AZ",AZ


#Budget Table

In [5454]:
Budget = pd.DataFrame(df['Budget (in$K)'])

Budget = Budget.drop_duplicates(['Type','Plan Year']).loc[:,['Type','Plan Year','Budget']]  #drop duplicates 
Budget.columns = ['budgetType','budgetPlanYear','budgetTotal']  #rename cols
Budget['budgetYearID'] = [np.random.randint(100, 1000) for _ in range(len(Budget))]  #assign ID for each budget type & plan year combo
Budget = Budget[['budgetYearID','budgetType','budgetPlanYear','budgetTotal']]  #rearrange columns

Budget.head()

,budgetYearID,budgetType,budgetPlanYear,budgetTotal
0,566,Design,2016,709000
3,333,Design,2017,1165000
5,674,Design,2018,395000
8,501,Design,2019,299000
12,137,Design,2020,275000


#AllocatedBudget Table

In [5456]:
AllocatedBudget = pd.DataFrame(df['Budget (in$K)'].drop(['Station Name','stationStateCode','Budget'],axis = 1))  #drop stationname, statecode,budget cols
AllocatedBudget.columns = ['budgetType','budgetPlanYear','stationCode','FY22','FY23','FY24','FY25','FY26']  #rename cols
AllocatedBudget = pd.melt(AllocatedBudget,id_vars = ['budgetType','budgetPlanYear','stationCode'], value_vars = ['FY22','FY23','FY24','FY25','FY26'],
                       value_name = 'allocatedBudget',var_name = 'allocatedBudgetYear') #unpivot table   

AllocatedBudget['allocatedBudgetYear'] = AllocatedBudget['allocatedBudgetYear'].str.replace('FY','20') #replace FY with 20
AllocatedBudget['allocatedBudget'] = AllocatedBudget['allocatedBudget']*1000 #multiple with 1000 to get the actual budget amount

AllocatedBudget.head()

,budgetType,budgetPlanYear,stationCode,allocatedBudgetYear,allocatedBudget
0,Design,2016,ATL,2022,658000.0
1,Design,2016,ELK,2022,459000.0
2,Design,2016,HFY,2022,1017000.0
3,Design,2017,ABE,2022,442000.0
4,Design,2017,KWD,2022,528000.0


#Route

In [5458]:
Route = pd.DataFrame(df['On-Time Performance (OTP)'])

Route = Route[['Type','Route']].drop_duplicates(['Type','Route'])  #drop duplicates
Route.columns = ['routeType','routeName']  #rename columns
unique_numbers = {name: f"{i:03}" for i, name in enumerate(Route['routeName'], start=100)}
Route['routeID'] =Route['routeName'].map(unique_numbers) #assign random numbers as identifiers for rout
Route = Route[['routeID','routeName','routeType']]  #rearrange cols

Route.head()

,routeID,routeName,routeType
0,100,Crescent,Long Distance
7,101,Southwest Chief,Long Distance
12,102,Sunset Limited,Long Distance
19,103,Texas Eagle,Long Distance
29,104,California Zephyr,Long Distance


#OTP Table

In [5460]:
OnTimePerformance= pd.DataFrame(df['On-Time Performance (OTP)']) #create otp df
OnTimePerformance = OnTimePerformance[['Route','Station','FY21 OTP','FY22 OTP','FY23 OTP']]

OnTimePerformance = pd.melt(OnTimePerformance, id_vars = ['Route','Station'],value_vars = ['FY21 OTP','FY22 OTP','FY23 OTP'],
                                                var_name = 'otpYear',value_name = 'otpValue')  #unpivot table
OnTimePerformance['otpYear'] = OnTimePerformance['otpYear'].str.replace('OTP','').str.replace('FY','20')  #rename values of year

#clean the station names manually & map it with the correct station names; in case of multiple stations in a city one of them is considered
OTPstations = OnTimePerformance ['Station'].drop_duplicates()
with pd.ExcelWriter('OTP-Station.xlsx') as writer:
    OTPstations.to_excel(writer, sheet_name='Stations', index=False)  #Write State to 'State' sheet

OTPstationsmap = pd.read_excel("C:/Users/prana/Desktop/DBMS/Project/OTP-Station-map.xlsx",sheet_name=None) #manual map file 
OnTimePerformance = pd.merge(OnTimePerformance,OTPstationsmap['Stations'],how = 'left')  #left join to get station name
OnTimePerformance = OnTimePerformance.dropna(subset = ['New Station'])  #drop if there are N/As
OnTimePerformance = OnTimePerformance[['Route','New Station','otpYear','otpValue']]
OnTimePerformance.columns = ['route','station','otpYear','otpValue']  #rename cols
OnTimePerformance = OnTimePerformance.drop_duplicates(['route','station','otpYear']) #drop duplicates (same station appearing twice)
OnTimePerformance.head()

,route,station,otpYear,otpValue
0,Crescent,"New York, NY – Moynihan Train Hall at Penn Sta...",2021,0.546
1,Crescent,"Washington, DC – Union Station",2021,0.546
2,Crescent,"Charlottesville, VA",2021,0.546
3,Crescent,"Charlotte, NC",2021,0.546
4,Crescent,"Atlanta, GA",2021,0.546


#Station Metrics table

In [5462]:
StationMetrics = pd.DataFrame(df['Ridership'])

StationMetrics = StationMetrics.loc[:,['City (Code) ','FY21','FY22','FY23']]
StationMetrics = pd.melt(StationMetrics,id_vars = 'City (Code) ',value_vars = ['FY21','FY22','FY23'],var_name = 'year',value_name = 'metric')  #unpivot
StationMetrics['stationMetric'] = 'Ridership'  #create metric column
StationMetrics.columns = ['station','stationMetricYear','stationMetricValue','stationMetric']  #rename columns
StationMetrics.stationMetricValue = StationMetrics.stationMetricValue.fillna(0)  #fill nulls with zero
StationMetrics['stationMetricYear'] = StationMetrics['stationMetricYear'].str.replace('FY','20')  #replace year values

StationMetrics.head()

,station,stationMetricYear,stationMetricValue,stationMetric
0,"Anniston, AL",2021,1948.0,Ridership
1,"Birmingham, AL",2021,14935.0,Ridership
2,"Tuscaloosa, AL",2021,3720.0,Ridership
3,"Benson, AZ",2021,1224.0,Ridership
4,"Flagstaff, AZ",2021,16693.0,Ridership


#State Metrics table

In [5464]:
Employment = pd.DataFrame( df['Employment'].loc[:,['State','FY21','FY22','FY23'] ] )
EmploymentWage = pd.DataFrame( df['Employment'].loc[:,['State','$FY21','$FY22','$FY23'] ] )

Employment = pd.melt(Employment,id_vars = 'State',value_vars = ['FY21','FY22','FY23'],value_name = 'stateMetricValue',var_name = 'stateMetricYear')
#unpivot table
Employment['stateMetric'] = 'Employment' #create metric name column

EmploymentWage = pd.melt(EmploymentWage,id_vars = 'State',value_vars = ['$FY21','$FY22','$FY23'],
                         value_name = 'stateMetricValue',var_name = 'stateMetricYear')  #unpivot table
EmploymentWage['stateMetric'] = 'EmploymentWage' #create metric name column

Procurement = pd.DataFrame(df['Procurement'])

Procurement = Procurement.loc[:,['State','FY21','FY22','FY23']]
Procurement = pd.melt(Procurement,id_vars = 'State',value_vars = ['FY21','FY22','FY23'],var_name = 'stateMetricYear',value_name = 'stateMetricValue')  #unpivot
Procurement['stateMetric'] = 'Procurement'  #create metric column
Procurement['stateMetricValue'] = Procurement['stateMetricValue'].fillna(0)  #fill NA values with zero

StateMetrics = pd.concat([Employment,EmploymentWage,Procurement],axis = 0)  #concat all three tables
StateMetrics = StateMetrics[StateMetrics.State.isna() == False]  #remove rows where state is null
StateMetrics['stateMetricYear'] = StateMetrics['stateMetricYear'].str.replace('$','').str.replace('FY','20')  #replace years with the numbers 
StateMetrics.head()

C:\Users\prana\AppData\Local\Temp\ipykernel_21856\1137653116.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Procurement['stateMetricValue'] = Procurement['stateMetricValue'].fillna(0)  #fill NA values with zero


,State,stateMetricYear,stateMetricValue,stateMetric
0,Alabama,2021,13,Employment
1,Arizona,2021,12,Employment
2,Arkansas,2021,26,Employment
3,California,2021,1842,Employment
4,Colorado,2021,58,Employment


#Join to fetch foreign keys

In [5466]:
AllocatedBudget = pd.merge(AllocatedBudget,Budget,how='left')  #fetch budgetYearID
AllocatedBudget = AllocatedBudget.loc[:,['budgetYearID','stationCode','allocatedBudgetYear','allocatedBudget']]
AllocatedBudget = AllocatedBudget[AllocatedBudget['allocatedBudget'] != "$                "]  #remove incorrect values
AllocatedBudget.head()

,budgetYearID,stationCode,allocatedBudgetYear,allocatedBudget
0,566,ATL,2022,658000.0
1,566,ELK,2022,459000.0
2,566,HFY,2022,1017000.0
3,333,ABE,2022,442000.0
4,333,KWD,2022,528000.0


In [5467]:
OnTimePerformance = pd.merge(OnTimePerformance,Route,left_on = 'route',right_on = 'routeName',how='left')  #fetch routeID
OnTimePerformance = pd.merge(OnTimePerformance,Station,left_on = 'station',right_on = 'stationName',how = 'left') #fetch State code
OnTimePerformance = OnTimePerformance.loc[:,['routeID','stationCode','otpYear','otpValue']]
OnTimePerformance.head()

,routeID,stationCode,otpYear,otpValue
0,100,NYP,2021,0.546
1,100,WAS,2021,0.546
2,100,CVS,2021,0.546
3,100,CLT,2021,0.546
4,100,ATL,2021,0.546


In [5468]:
StationMetrics = pd.merge(StationMetrics,Station,left_on = 'station',right_on = 'stationName',how='left')  #fetch stationID
StationMetrics = StationMetrics.loc[:,['stationCode','stationMetric','stationMetricYear','stationMetricValue']] #rearrange columns

StationMetrics.head()

,stationCode,stationMetric,stationMetricYear,stationMetricValue
0,ATN,Ridership,2021,1948.0
1,BHM,Ridership,2021,14935.0
2,TCL,Ridership,2021,3720.0
3,BEN,Ridership,2021,1224.0
4,FLG,Ridership,2021,16693.0


In [5469]:
StateMetrics = pd.merge(StateMetrics,State, left_on = 'State',right_on = 'stateName',how='left')  #fetch stationcode
StateMetrics = StateMetrics.loc[:,['stateCode','stateMetric','stateMetricYear','stateMetricValue']] #rearrange columns

StateMetrics.head()

,stateCode,stateMetric,stateMetricYear,stateMetricValue
0,AL,Employment,2021,13
1,AZ,Employment,2021,12
2,AR,Employment,2021,26
3,CA,Employment,2021,1842
4,CO,Employment,2021,58


#Changes to station table

In [5470]:
Station = Station.drop_duplicates('stationCode')  #drop duplicate station codes that might appear since same code can have multiple names
Station['stationName'] = Station['stationName'].str.replace(r',.*?(–|$)', '', regex=True) #reformat the stationName 
Station.head()

,stationCode,stationName,stateCode
0,ATN,Anniston,AL
1,BHM,Birmingham,AL
2,TCL,Tuscaloosa,AL
3,BEN,Benson,AZ
4,FLG,Flagstaff,AZ


#Export data into excel

In [5487]:
OnTimePerformance.head()

with pd.ExcelWriter('Amtrak-CleanedData-Final.xlsx') as writer:
    State.to_excel(writer, sheet_name='State', index=False)  # Write State to 'State' sheet
    Station.to_excel(writer, sheet_name='Station', index=False)  # Write Station to 'Station' sheet    
    StateMetrics.to_excel(writer, sheet_name='StateMetrics', index=False)  # Write StateMetrics to 'StateMetrics' sheet
    StationMetrics.to_excel(writer, sheet_name='StationMetrics', index=False)  # Write StationMetrics to 'StationMetrics' sheet
    Route.to_excel(writer, sheet_name='Route', index=False)  # Write Route to 'Route' sheet
    OnTimePerformance.to_excel(writer, sheet_name='OnTimePerformance', index=False)  # Write OnTimePerformance to 'OnTimePerformance' sheet
    Budget.to_excel(writer, sheet_name='Budget', index=False)  # Write Budget to 'Budget' sheet
    AllocatedBudget.to_excel(writer, sheet_name='AllocatedBudget', index=False)  # Write BudgetValue to 'BudgetValue' sheet

In [1879]:
#PreProcess
#Overall ridership for two lakeland stations - for both divide in proportion of 2021 values in excel file before loading
#changed Lakeland station names since there are two for better identification
#incorrect code for Tucson, AZ corrected in the input file
#incorrect code for winter park in budget tab is now corrected